<a href="https://colab.research.google.com/github/zaellis/sscs-ose-code-a-chip.github.io/blob/main/VLSI24/submitted_notebooks/SJSystolicArray/SystolicArray.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Wishbone ASCON with OpenLane

```
Copyright 2023 SiliconJackets
SPDX-License-Identifier: GPL-3.0-or-later
```

Running a 3x3 systolic array design inspired by [EYERISS](https://courses.cs.washington.edu/courses/cse550/21au/papers/CSE550.Eyeriss.pdf) design thru the [OpenLane](https://github.com/The-OpenROAD-Project/OpenLane/) GDS to RTL flow targeting the [open source SKY130 PDK](https://github.com/google/skywater-pdk/) with the addition of dual-port RAM macros generated by [OpenRAM](https://github.com/VLSIDA/OpenRAM).

|Name|Affiliation| Email |IEEE Member|SSCS Member|
|:--:|:----------:|:----------:|:----------:|:----------:|
|Zachary Ellis|Georgia Institute of Technology|zellis7@gatech.edu|Yes|Yes|
|Nealson Li|Georgia Institute of Technology|nealson@gatech.edu|Yes|Yes|
|Addison Elliott|Georgia Institute of Technology|addisonelliott@gatech.edu|Yes|Yes|
|Zeyan Wu|Georgia Institute of Technology|zwu477@gatech.edu|Yes|Yes|

In [1]:
#@title Install dependencies {display-mode: "form"}
#@markdown - Click the ▷ button to setup the digital design environment based on [conda-eda](https://github.com/hdl/conda-eda).

openlane_version = 'latest' #@param {type:"string"}
open_pdks_version = 'latest' #@param {type:"string"}

if openlane_version == 'latest':
  openlane_version = ''
if open_pdks_version == 'latest':
  open_pdks_version = ''

import os
import pathlib

!curl -Ls https://micro.mamba.pm/api/micromamba/linux-64/latest | tar -xj bin/micromamba
conda_prefix_path = pathlib.Path('conda-env')
CONDA_PREFIX = str(conda_prefix_path.resolve())
!bin/micromamba create --yes --prefix $CONDA_PREFIX
!echo 'python ==3.7*' >> {CONDA_PREFIX}/conda-meta/pinned
!CI=0 bin/micromamba install --yes --prefix $CONDA_PREFIX \
                     --channel litex-hub \
                     --channel main \
                     openlane={openlane_version} \
                     open_pdks.sky130a={open_pdks_version}
!python -m pip install gdstk gdstk
!python -m pip install libparse libparse
PATH = os.environ['PATH']
%env CONDA_PREFIX={CONDA_PREFIX}
%env PATH={CONDA_PREFIX}/bin:{PATH}



Empty environment created at prefix: /content/conda-env

Pinned packages:
  - python 3.7*


Transaction

  Prefix: /content/conda-env

  Updating specs:

   - openlane=*
   - open_pdks.sky130a=*


  Package                                                Version  Build                 Channel         Size
──────────────────────────────────────────────────────────────────────────────────────────────────────────────
  Install:
──────────────────────────────────────────────────────────────────────────────────────────────────────────────

  + open_pdks.sky130a                         1.0.471_0_g97d0844  20240223_100318       litex-hub     Cached
  + _libgcc_mutex                                            0.1  main                  main          Cached
  + libstdcxx-ng                                          11.2.0  h1234567_1            main          Cached
  + ld_impl_linux-64                                        2.38  h1181459_1            main          Cached
  + ca-certificates     

In [1]:
%cd /tmp
! git clone -b v1.2.8 https://github.com/VLSIDA/OpenRAM > /dev/null 2>&1
%cd /tmp/OpenRAM
! ./install_conda.sh #> /dev/null 2>&1
! python -m pip install -r requirements.txt > /dev/null 2>&1
%env OPENRAM_ROOT=/tmp/OpenRAM
%env OPENRAM_HOME="/tmp/OpenRAM/compiler"
%env OPENRAM_TECH="/tmp/OpenRAM/technology"
%env PYTHONPATH=$OPENRAM_HOME
%env PYTHONPATH="$OPENRAM_HOME:$OPENRAM_TECH/sky130:$OPENRAM_TECH/sky130/custom"
! source miniconda/bin/activate && make pdk > /dev/null 2>&1
! make install > /dev/null 2>&1

/tmp
/tmp/OpenRAM
^C
env: OPENRAM_ROOT=/tmp/OpenRAM
env: OPENRAM_HOME="/tmp/OpenRAM/compiler"
env: OPENRAM_TECH="/tmp/OpenRAM/technology"
env: PYTHONPATH=$OPENRAM_HOME
env: PYTHONPATH="$OPENRAM_HOME:$OPENRAM_TECH/sky130:$OPENRAM_TECH/sky130/custom"
Traceback (most recent call last):
  File "/tmp/OpenRAM/miniconda/bin/conda", line 12, in <module>
    from conda.cli import main
  File "/tmp/OpenRAM/miniconda/lib/python3.8/site-packages/conda/cli/__init__.py", line 3, in <module>
    from .main import main  # NOQA
  File "/tmp/OpenRAM/miniconda/lib/python3.8/site-packages/conda/cli/main.py", line 34, in <module>
    from .conda_argparse import generate_parser
  File "/tmp/OpenRAM/miniconda/lib/python3.8/site-packages/conda/cli/conda_argparse.py", line 27, in <module>
    from ..base.context import context
  File "/tmp/OpenRAM/miniconda/lib/python3.8/site-packages/conda/base/context.py", line 9, in <module>
    from typing import Optional
  File "<frozen importlib._bootstrap>", line 991, i

In [ ]:
%%writefile macros/sram_configs/myconfig.py

"""
Dual port (1 read/write + 1 read only) 1 kbytes SRAM with byte write.
"""
word_size = 8

num_words = 16

human_byte_size = "{:.0f}kbytes".format((word_size * num_words)/1024/8)

# Allow byte writes
write_size = 2 # Bits

# Dual port
num_rw_ports = 1
num_r_ports = 1
num_w_ports = 0
ports_human = '1rw1r'

import os
exec(open(os.path.join(os.path.dirname(__file__), 'sky130_sram_common.py')).read())

In [ ]:

config_file = "macros/sram_configs/myconfig.py"


#@markdown ---
#@markdown ### Number of bits for each memory word:
word_size = 8 #@param {type:"raw"}

#@markdown ---
#@markdown ### Total number of memory words:
num_words = 16 #@param {type:"raw"}

#@markdown ---
#@markdown ### Lowest number of writable bits in one write cylce:
write_size = 2 #@param {type:"raw"}

#@markdown ---
#@markdown ### Number of Read/Write ports:
num_rw_ports = 1 #@param {type:"raw"}

#@markdown ---
#@markdown ### Number of Read only ports:
num_r_ports = 1 #@param {type:"raw"}

#@markdown ---
#@markdown ### Number of Write only ports:
num_w_ports = 0 #@param {type:"raw"}

#@markdown ---



! sed -i "s/^word_size.*$/word_size = $word_size/" $config_file
! sed -i "s/^num_words.*$/num_words = $num_words/" $config_file
! sed -i "s/^write_size.*$/write_size = $write_size/" $config_file
! sed -i "s/^num_rw_ports.*$/num_rw_ports = $num_rw_ports/" $config_file
! sed -i "s/^num_r_ports.*$/num_r_ports = $num_r_ports/" $config_file
! sed -i "s/^num_w_ports.*$/num_w_ports = $num_w_ports/" $config_file
! sed -i "s/^FIXME.*$//" $config_file

In [ ]:
! cd macros/ && make myconfig.ok > /dev/null 2>&1

In [6]:
%%writefile config.py
word_size = 32 # Bits
num_words = 32
human_byte_size = "{:.0f}kbytes".format((word_size * num_words)/1024/8)

# Allow byte writes
write_size = 32 # Bits

# Dual port
num_rw_ports = 0
num_r_ports = 1
num_w_ports = 1
ports = '1r1w'

tech_name = 'sky130'
nominal_corner_only = True

route_supplies = 'ring'
check_lvsdrc = True
uniquify = True

output_name = f'{tech_name}_sram_{ports}_{word_size}x{num_words}_{write_size}'
output_path = '.'

Overwriting config.py


In [7]:
#@markdown Run OpenRAM
!python3 $OPENRAM_HOME/../sram_compiler.py config.py

|==============================================================================|
|=========                      OpenRAM v1.2.48                       =========|
|=========                                                            =========|
|=========               VLSI Design and Automation Lab               =========|
|=========        Computer Science and Engineering Department         =========|
|=========            University of California Santa Cruz             =========|
|=========                                                            =========|
|=========          Usage help: openram-user-group@ucsc.edu           =========|
|=========        Development help: openram-dev-group@ucsc.edu        =========|
|=========                See LICENSE for license info                =========|
|==============================================================================|
** Start: 03/20/2024 22:02:34
Technology: sky130
Total size: 1024 bits
Word size: 32
Words: 32
Banks: 1
RW po